# Complete Example
Here is a complete example showing how to implement a simple operator using the taichi custom operator:

In [1]:
import jax
import jax.numpy as jnp
import taichi as ti

import braintaichi as bti
import brainstate as bst


@ti.func
def get_weight(weight: ti.types.ndarray()) -> ti.f32:
  return weight[None]


@ti.func
def update_output(out: ti.types.ndarray(), index: ti.i32, weight_val: ti.f32):
  out[index] += weight_val


@ti.kernel
def event_ell_cpu(indices: ti.types.ndarray(),
                  vector: ti.types.ndarray(),
                  weight: ti.types.ndarray(),
                  out: ti.types.ndarray()):
  weight_val = get_weight(weight)
  num_rows, num_cols = indices.shape
  ti.loop_config(serialize=True)
  for i in range(num_rows):
    if vector[i]:
      for j in range(num_cols):
        update_output(out, indices[i, j], weight_val)

@ti.kernel
def event_ell_gpu(indices: ti.types.ndarray(),
                  vector: ti.types.ndarray(), 
                  weight: ti.types.ndarray(), 
                  out: ti.types.ndarray()):
  weight_val = get_weight(weight)
  num_rows, num_cols = indices.shape
  for i in range(num_rows):
    if vector[i]:
      for j in range(num_cols):
        update_output(out, indices[i, j], weight_val)

prim = bti.XLACustomOp(cpu_kernel=event_ell_cpu, gpu_kernel=event_ell_gpu)


def test_taichi_op_register():
  s = 1000
  indices = bst.random.randint(0, s, (s, 1000))
  vector = bst.random.rand(s) < 0.1

  out = prim(indices, vector, 1.0, outs=[jax.ShapeDtypeStruct((s,), dtype=jnp.float32)])

  out = prim(indices, vector, 1.0, outs=[jax.ShapeDtypeStruct((s,), dtype=jnp.float32)])

  print(out)

test_taichi_op_register()

[Taichi] version 1.7.2, llvm 15.0.1, commit 0131dce9, win, python 3.10.13
[Array([122., 106., 119., 123., 102., 125., 116., 113., 121., 106., 118.,
       111., 113., 115., 114., 118., 119., 109., 112., 105., 113., 121.,
       122., 113., 108., 116., 104.,  89., 108., 111., 110., 131., 119.,
       125., 117.,  98., 107., 118., 112., 121., 121., 119., 119., 122.,
       102., 126., 109., 116., 128., 113., 110., 119., 109., 110., 101.,
       128., 124., 112., 110., 132.,  98., 109., 111., 115., 134., 126.,
       125., 102., 133., 118., 112., 123., 123., 113., 119., 115., 123.,
       121., 128., 113., 123., 129., 108., 132., 109., 115.,  81., 125.,
       111., 132., 116., 127., 102., 126., 109., 123., 123., 106., 112.,
       131., 106., 117., 107., 112., 133., 130., 110., 109., 142., 101.,
       120., 111., 123., 126., 130., 139., 116., 128., 101., 118., 120.,
       123., 104., 119., 126., 113., 111., 113., 102., 110., 120., 133.,
       134., 111., 101., 120., 116., 106., 111., 

# More Examples
For more examples, please refer to:
- [braintaichi/_eventop/_event_csrmv.py](https://github.com/chaoming0625/braintaichi/blob/main/braintaichi/_eventop/_event_csrmv.py)
- [braintaichi/_sparseop/_sparse_csrmv.py](https://github.com/chaoming0625/braintaichi/blob/main/braintaichi/_sparseop/_sparse_csrmv.py)
- [braintaichi/_jitconnop/_jit_event_csrmv.py](https://github.com/chaoming0625/braintaichi/blob/main/braintaichi/_jitconnop/_jit_event_csrmv.py)
- [braintaichi/_jitconnop/_jit_csrmv.py](https://github.com/chaoming0625/braintaichi/blob/main/braintaichi/_jitconnop/_jit_csrmv.py)